In [1]:
import pandas as pd
import glob
import numpy as np
from PIL import Image
import sys
import keras
import os
import random
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(categorical_features=[1])

Using TensorFlow backend.


In [2]:
batch_size=64
epochs=20
num_classes=3

# Load the npy file saved from processing data
# Onehotencode the labeled csv file, and transform into numpy array

train_data=np.load('train_data.npy')
df=pd.read_csv("train_labels.csv")
y_train=onehotencoder.fit_transform(df).toarray()
y_train=np.delete(y_train,3,axis=1)
train_label=y_train[np.arange(6738)]

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [3]:
# Load the test-set image and label 

test_data=np.load(test_data.npy)
df1=pd.read_csv("test_labels.csv")
y_test=onehotencoder.fit_transform(df1).toarray()
y_test=np.delete(y_test,3,axis=1)
test_label=y_test[np.arange(3000)]

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers

# Define a CNN classifier network composed of a sequence of layers
model = Sequential()

# The input layer accepts an image and applies a convolution that uses 24 3x3 filters and a rectified linear unit activation function
# Next add a max pooling layer with a 2*2 patch

model.add(Conv2D(24, (3,3), input_shape=(128,173,4)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# Add more layers, including dropout layer that prevent over-fitting
model.add(Conv2D(48, (3,3)))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Conv2D(96, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dropout(0.3))

# Now we'll flatten the feature maps and generate an output layer with a predicted probability for each class

model.add(Dense(num_classes))
model.add(Activation("softmax"))

# With the layers defined, we can now compile the model for categorical (multi-class) classification
opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 171, 24)      888       
_________________________________________________________________
activation_5 (Activation)    (None, 126, 171, 24)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 85, 24)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 83, 48)        10416     
_________________________________________________________________
activation_6 (Activation)    (None, 61, 83, 48)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 61, 83, 48)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 59, 81, 96)        41568     
__________

In [5]:
model.fit(train_data, train_label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_data, test_label))
score = model.evaluate(test_data, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 6738 samples, validate on 3000 samples
Epoch 1/20
6738/6738 [==============================] - 346s 51ms/step - loss: 1.2002 - acc: 0.4061 - val_loss: 1.0525 - val_acc: 0.4520
Epoch 2/20
6738/6738 [==============================] - 372s 55ms/step - loss: 0.9700 - acc: 0.5255 - val_loss: 0.9680 - val_acc: 0.5063
Epoch 3/20
6738/6738 [==============================] - 379s 56ms/step - loss: 0.8676 - acc: 0.6060 - val_loss: 0.8883 - val_acc: 0.5813
Epoch 4/20
6738/6738 [==============================] - 382s 57ms/step - loss: 0.7989 - acc: 0.6457 - val_loss: 0.8200 - val_acc: 0.6287
Epoch 5/20
6738/6738 [==============================] - 382s 57ms/step - loss: 0.7176 - acc: 0.6949 - val_loss: 0.7470 - val_acc: 0.6727
Epoch 6/20
6738/6738 [==============================] - 384s 57ms/step - loss: 0.6306 - acc: 0.7462 - val_loss: 0.6777 - val_acc: 0.7147
Epoch 7/20
6738/6738 [==============================] - 428s 63ms/step - loss: 0.5725 - acc: 0.7707 - val_loss: 0.5859 - val_acc: 

In [6]:
# Save the model

model.save('optimized_model')